In [2]:
from ndtools import staged_max_flow as smf

from pathlib import Path
import json

from tsum import tsum
import torch

# Reliability analysis of benchmark gas supply plant

## Case 1: Original system

### Load data

In [6]:
DATASET = Path(r"data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))


In [16]:
def s_fun(comps_st):
    flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, nodes, edges, probs_dict, target_flow = 0.5 )

    sys_st = 1 if sys_st_str == 's' else 0
    return flow, sys_st, None

row_names = list(probs_dict.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

### Get rules for system event

In [ ]:
sys_surv_st = 1 

result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-5,
    output_dir="tsum_res"
) 

## Case 2: Addition Importance Measure

### Get rules for system event with each component added

In [ ]:
for x in probs_dict.keys():
    print(f"Adding component: {x}")

    new_nodes, new_edges, new_probs = smf.add_a_component(x, nodes, edges, probs_dict)

    def s_fun(comps_st):
        flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, new_nodes, new_edges, new_probs, target_flow = 0.5 )

        sys_st = 1 if sys_st_str == 's' else 0
        return flow, sys_st, None
    
    row_names = list(new_probs.keys())
    n_state = 2  # binary states: 0, 1
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    probs = [[new_probs[n]['0']['p'], new_probs[n]['1']['p']] for n in row_names]
    probs = torch.tensor(probs, dtype=torch.float32, device=device)

    # get rules
    sys_surv_st = 1
    
    result = tsum.run_rule_extraction_by_mcs(
        # Problem-specific callables / data
        sfun=s_fun,
        probs=probs,
        row_names=row_names,
        n_state=n_state,
        sys_surv_st=sys_surv_st,
        unk_prob_thres = 1e-5,
        output_dir=rf"tsum_res_aim/{x}"
    ) 

Adding component: x1
---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 0.5. Total samples: 100000.
New rule (No. of conditions: 23): {'x1': ('>=', 1), 'x6': ('>=', 1), 'x9': ('>=', 1), 'x28': ('>=', 1), 'x30': ('>=', 1), 'x31': ('>=', 1), 'x33': ('>=', 1), 'x35': ('>=', 1), 'x36': ('>=', 1), 'x38': ('>=', 1), 'x40': ('>=', 1), 'x41': ('>=', 1), 'x46': ('>=', 1), 'x47': ('>=', 1), 'x48': ('>=', 1), 'x61': ('>=', 1), 'x62': ('>=', 1), 'x64': ('>=', 1), 'x66': ('>=', 1), 'x82': ('>=', 1), 'x84': ('>=', 1), 'x85': ('>=', 1), 'x87': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [0.5]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 1, Failure rules: 0
Survival sample found from sampling.
No. of existin

## Case 3: Deactivation Importance Measure

### Get rules for system event with each component deactivated

In [ ]:
for x in probs_dict.keys():
    print(f"Deactivating component: {x}")

    new_nodes, new_edges, new_probs = smf.deactivate_a_component(x, nodes, edges, probs_dict)

    def s_fun(comps_st):
        flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, new_nodes, new_edges, new_probs, target_flow = 0.5 )

        sys_st = 1 if sys_st_str == 's' else 0
        return flow, sys_st, None
    
    row_names = list(new_probs.keys())
    n_state = 2  # binary states: 0, 1
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    probs = [[new_probs[n]['0']['p'], new_probs[n]['1']['p']] for n in row_names]
    probs = torch.tensor(probs, dtype=torch.float32, device=device)

    # get rules
    sys_surv_st = 1
    
    result = tsum.run_rule_extraction_by_mcs(
        # Problem-specific callables / data
        sfun=s_fun,
        probs=probs,
        row_names=row_names,
        n_state=n_state,
        sys_surv_st=sys_surv_st,
        unk_prob_thres = 1e-5,
        output_dir=rf"tsum_res_dim/{x}"
    ) 

Deactivating component: x1
---
Round: 1, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 0.5. Total samples: 100000.
New rule (No. of conditions: 21): {'x2': ('>=', 1), 'x3': ('>=', 1), 'x5': ('>=', 1), 'x28': ('>=', 1), 'x30': ('>=', 1), 'x32': ('>=', 1), 'x33': ('>=', 1), 'x34': ('>=', 1), 'x35': ('>=', 1), 'x36': ('>=', 1), 'x38': ('>=', 1), 'x39': ('>=', 1), 'x46': ('>=', 1), 'x61': ('>=', 1), 'x62': ('>=', 1), 'x64': ('>=', 1), 'x66': ('>=', 1), 'x82': ('>=', 1), 'x84': ('>=', 1), 'x85': ('>=', 1), 'x87': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [0.5]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 1, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule a